In [11]:
from pyrosetta import *

#Removing -mute all is a good idea for debugging
init("-beta_nov16_cart -ignore_unrecognized_res -mute all -ex1 -ex2aro") 


Found rosetta database at: /work/dadriano/.virtualenvs/may2017_pyrosetta_newDigs/lib/python2.7/site-packages/pyrosetta-4.0-py2.7.egg/database; using it....
PyRosetta-4 2016 [Rosetta 2016 unknown:fdd9e71c3a71b1bbd5e9e6f21327432d10df240c 2017-03-27 11:09:48 -0400] retrieved from: git@github.com:RosettaCommons/main.git
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.



In [12]:
# First do some initial setup

pose = pose_from_pdb("src_pdbs/LPC_dryRun_3lB6A_Mf14aa_order00029_ndx6_c189_2cig12p_0001_-41.8_re1_6_EEHEE_0883_0001_0002.pdb")
save_pose = pose.clone()

scorefxn = get_fa_scorefxn()


In [15]:
# First we're going to do FastRelax because it has the easiest setup

# Initialize the FastRelax (mover) with iterations
fast_relax = rosetta.protocols.relax.FastRelax(1)
fast_relax.set_scorefxn(scorefxn)
fast_relax.cartesian(True)
#fast_relax.constrain_coords(True)

# Optionally give it a movemap
#fast_relax.set_movemap(mm)

In [ ]:
# Watch out, this takes a long time
print("Before score: %.3f"%scorefxn(pose))

fast_relax.apply(pose)

print(" After score: %.3f"%scorefxn(pose))

pose = save_pose.clone()

In [16]:
# Using the Packer and TaskOperations

# Now we're going to do some interface design. The idea is to design
#  the interface on chain B while repacking the interface on
#  chain A.

# First we need to setup some residue selectors

chain_A = rosetta.core.select.residue_selector.ChainSelector("A")
chain_B = rosetta.core.select.residue_selector.ChainSelector("B")

# Neighbors near chain_A, within 7 A, but not chain_A
design_interface = rosetta.core.select.residue_selector.NeighborhoodResidueSelector(
    chain_A, 7, False )

# Neighbors near chain_B, within 7 A, but not chain_B
repack_interface = rosetta.core.select.residue_selector.NeighborhoodResidueSelector(
    chain_B, 7, False )

# The combined set that we're going to work on
do_something_set = rosetta.core.select.residue_selector.OrResidueSelector(
    design_interface, repack_interface)


# The neighborhood residue selectors require the pose to be scored so let's make sure
#  that's true.
scorefxn(pose)


# Here's a one-liner to show a residue selector
print("Design interface")
print(rosetta.core.select.get_residues_from_subset(design_interface.apply(pose)))
print("Repack interface")
print(rosetta.core.select.get_residues_from_subset(repack_interface.apply(pose)))

Design interface
vector1_unsigned_long[111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 146, 148, 150]
Repack interface
vector1_unsigned_long[1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 40, 41, 44, 57, 58, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 107, 108, 109]


In [17]:
# Now lets see how to set up task operations

# The task factory accepts all the task operations
tf = rosetta.core.pack.task.TaskFactory()

# These three are pretty standard
tf.push_back(rosetta.core.pack.task.operation.InitializeFromCommandline())
tf.push_back(rosetta.core.pack.task.operation.IncludeCurrent())
tf.push_back(rosetta.core.pack.task.operation.NoRepackDisulfides())


# If we weren't going to be designing, we would use this too
# tf.push_back(rosetta.core.pack.task.operation.RestrictToRepacking())

# disable design on the only repack part
tf.push_back(rosetta.core.pack.task.operation.OperateOnResidueSubset(
    rosetta.core.pack.task.operation.RestrictToRepackingRLT(), repack_interface ))

# don't repack the rest of the protein
tf.push_back(rosetta.core.pack.task.operation.OperateOnResidueSubset(
    rosetta.core.pack.task.operation.PreventRepackingRLT(), do_something_set, True ))
                                                                     #        ^^^^
                                                                     #invert selection

# Include a ResFile
# tf.push_back(rosetta.core.pack.task.operation.ReadResfile("your/res/file"))

# Going to use only polar residues because I heard hydrogen bonds are good
aa_to_design = rosetta.core.pack.task.operation.RestrictAbsentCanonicalAASRLT()
aa_to_design.aas_to_keep( "STQN" )

tf.push_back(rosetta.core.pack.task.operation.OperateOnResidueSubset(
    aa_to_design, design_interface ))


# Have to convert the task factory into a PackerTask
task = tf.create_task_and_apply_taskoperations( pose )


In [6]:
# Let's see what the PackerTask looks like
print(task)

#Packer_Task

resid	pack?	design?	allowed_aas
1	TRUE	FALSE	PRO:NtermProteinFull
2	TRUE	FALSE	PRO
3	TRUE	FALSE	SER
4	TRUE	FALSE	THR
5	TRUE	FALSE	ALA
6	TRUE	FALSE	LEU
7	TRUE	FALSE	ARG
8	TRUE	FALSE	GLU
9	TRUE	FALSE	LEU
10	FALSE	FALSE	
11	TRUE	FALSE	GLU
12	TRUE	FALSE	GLU
13	FALSE	FALSE	
14	FALSE	FALSE	
15	FALSE	FALSE	
16	FALSE	FALSE	
17	FALSE	FALSE	
18	FALSE	FALSE	
19	FALSE	FALSE	
20	FALSE	FALSE	
21	FALSE	FALSE	
22	FALSE	FALSE	
23	FALSE	FALSE	
24	FALSE	FALSE	
25	FALSE	FALSE	
26	FALSE	FALSE	
27	FALSE	FALSE	
28	FALSE	FALSE	
29	FALSE	FALSE	
30	FALSE	FALSE	
31	FALSE	FALSE	
32	FALSE	FALSE	
33	FALSE	FALSE	
34	FALSE	FALSE	
35	FALSE	FALSE	
36	FALSE	FALSE	
37	FALSE	FALSE	
38	FALSE	FALSE	
39	FALSE	FALSE	
40	TRUE	FALSE	TYR
41	FALSE	FALSE	
42	FALSE	FALSE	
43	FALSE	FALSE	
44	TRUE	FALSE	LEU
45	FALSE	FALSE	
46	FALSE	FALSE	
47	FALSE	FALSE	
48	FALSE	FALSE	
49	FALSE	FALSE	
50	FALSE	FALSE	
51	FALSE	FALSE	
52	FALSE	FALSE	
53	FALSE	FALSE	
54	FALSE	FALSE	
55	FALSE	FALSE	
56	FALSE	FALSE	
57	TRUE	FALSE	GLU
58	TRU

In [7]:
# Now it's finally time to use the packer

# The packer takes a scorefunction and a PackerTask
pack_mover = rosetta.protocols.simple_moves.PackRotamersMover(scorefxn, task)

print("Before score: %.3f"%scorefxn(pose))

pack_mover.apply(pose)

print(" After score: %.3f"%scorefxn(pose))



Before score: 56.045
 After score: 78.874


In [8]:
# Let's compare sequences to see what happened:
print save_pose.sequence()
print pose.sequence()


PPSTALRELIEELVNITQNQKAPLCNGSMVWSINLTAGMYCAALESLINVSGCSAIEKTQRMLSGFCPHKVSAGQFSSLHVRDTKIEVAQFVKDLLLHLKKLFREGRFNPTAKLQDDEAEWYSRDAANAAAALAAQTRVETRNDEIRSDS
PPSTALRELIEELVNITQNQKAPLCNGSMVWSINLTAGMYCAALESLINVSGCSAIEKTQRMLSGFCPHKVSAGQFSSLHVRDTKIEVAQFVKDLLLHLKKLFREGRFNPTSTSQSSTSTTSSSNSSNSSSSNSNQTRVETRNDETRSDT


In [9]:
# Now let's use the minimizer to clean up the packing

# Movemap the moves everything
mm = rosetta.core.kinematics.MoveMap()
mm.set_bb(True)
mm.set_chi(True)
mm.set_jump(True)


# vector1 of true/false for each residue in the pose
subset_to_minimize = do_something_set.apply(pose)

for i in range(1, pose.size() + 1):
    if ( not subset_to_minimize[i] ):
        mm.set_bb(i, False)
        mm.set_chi(i, False)


min_mover = rosetta.protocols.simple_moves.MinMover()
min_mover.cartesian(True)
min_mover.movemap(mm)
min_mover.tolerance(1E-4)
min_mover.score_function(scorefxn)

print("Before score: %.3f"%scorefxn(pose))
min_mover.apply(pose)
print(" After score: %.3f"%scorefxn(pose))


# Let's dump the pdb so we can save this awesome design
pose.dump_pdb("awesome_design.pdb")





Before score: 78.874
 After score: 1.443


True

In [ ]:
# I want to test something Daniel told me
# He said that you don't need a virtual root for cartesian minimization
# even if you use coordinate constraints


# Set up a coordinate constraint to pull the first atom out to -1000, -1000, -1000
harmonic = rosetta.core.scoring.func.HarmonicFunc( 0, 0.0001 )

constrained_atom = rosetta.core.id.AtomID( pose.residue(1).atom_index("N"), 1 )
root_atom = rosetta.core.id.AtomID( 1, pose.fold_tree().root() )

point = rosetta.numeric.xyzVector_double_t( -1000, -1000, -1000 )


# syntax is: constrained atom, atom that won't move, xyz, func
cst = rosetta.core.scoring.constraints.CoordinateConstraint(
        constrained_atom, root_atom, point, harmonic)


# Really we only need jumps turned on, but it doesn't hurt to have it all on
mm = rosetta.core.kinematics.MoveMap()
mm.set_bb(True)
mm.set_chi(True)
mm.set_jump(True)

min_mover = rosetta.protocols.simple_moves.MinMover()
min_mover.cartesian(True)
min_mover.movemap(mm)
min_mover.tolerance(1E-4)
min_mover.score_function(scorefxn)


print("Before: " + str(pose.residue(1).atom(1)))

pose.add_constraint(cst)
min_mover.apply(pose)
pose.remove_constraints()

print("After:" + str(pose.residue(1).atom(1)))


# What do you know, Daniel was right

pose = save_pose.clone()

